In [1]:
%load_ext autoreload
%autoreload 2
%xmode Plain

Exception reporting mode: Plain


In [2]:
import pandas as pd
import plotly
import plotly.express as px 
import dtale
import numpy as np
import random
from bazaraki import utils
from tqdm import tqdm
import swifter
from pathlib import Path

In [3]:
tqdm.pandas()
pd.set_option('display.max_rows', 100)  # Disable row limit
pd.set_option('display.max_columns', 60)  # Disable column limit
pd.set_option('display.width', 20)  # Disable line width limit
pd.set_option('display.max_colwidth', 100)  # Disable column width limit
pd.set_option('display.precision', 2)  
pd.set_option('display.float_format', '{:.4f}'.format)  
np.random.seed(42)
random.seed(42)
pd.options.plotting.backend = "plotly"
plotly.io.renderers.default = "notebook_connected"
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
df = pd.read_json("output/20241212_183425_real-estate-to-rent_real-estate-for-sale.jsonl", lines=True)

In [5]:
df.iloc[0].T

url                                                 https://www.bazaraki.com/adv/5415213_4-bedroom-detached-house-to-rent/
title                                                                                     4-bedroom detached house to rent
price                                                                                                                 1650
original_price                                                                                                         NaN
price_per_sqm                                                                                                          NaN
location                                                                                               Larnaca, Dromolaxia
posted                                                                                                     Yesterday 20:57
ad_id                                                                                                              5415213
reference_number

In [6]:
df["price_sqm"] = df.price / (df["Property area"]).round(2)

In [7]:
def add_city_disctrict_cols(df):
    df[["city", "district"]] = df.location.str.split(",", expand=True)
    return df
df = add_city_disctrict_cols(df)


In [8]:
df = utils.filter_in(df, "`Property area` > 10 and `Property area` < 300")

removing 9658 rows


In [9]:
saledf = df[df.cat1.isin(['Apartments, flats for sale'])] # 'Houses for sale', 

rentdf = df[df.cat1.isin(['Apartments, flats to rent'])] # 'Houses to rent', 


In [10]:

import math  


def haversine(lat1, lon1, lat2, lon2):  
    """  
    Calculate the great-circle distance between two points on the Earth (specified in decimal degrees).  
    Returns the distance in kilometers.  
    """  
    # Convert decimal degrees to radians  
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])  
    # Haversine formula  
    dlat = lat2 - lat1  
    dlon = lon2 - lon1  
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2  
    c = 2 * math.asin(math.sqrt(a))  
    # Radius of Earth in kilometers (mean radius)  
    r = 6371.0  
    return c * r  

lat1, lon1 = 52.5200, 13.4050  # Berlin  
lat2, lon2 = 52.5205, 13.4095  # Nearby point in Berlin  
print(haversine(lat1, lon1, lat2, lon2))


0.3095045367196025


In [11]:

def rent_filter(rentdf, radius_km, sale_row):
    return rentdf[rentdf.apply(lambda r: haversine(r.lat, r.lng, sale_row.lat, sale_row.lng), axis=1) < radius_km]

def rent_finder(rentdf, radius_km, verbose, sale_row):
    if verbose:
        print(sale_row.url)
    # Step 1: Calculate the bounding box
    lat_diff = radius_km / 111.32  # Approx. 1 degree latitude = 111.32 km
    lng_diff = radius_km / (111.32 * np.cos(np.radians(sale_row.lat)))  # Adjust for latitude

    # Bounding box
    min_lat = sale_row.lat - lat_diff
    max_lat = sale_row.lat + lat_diff
    min_lng = sale_row.lng - lng_diff
    max_lng = sale_row.lng + lng_diff
    
    # Step 2: Filter rows within the bounding box
    filtered_df = rentdf[(rentdf['lat'] >= min_lat) & (rentdf['lat'] <= max_lat) & (rentdf['lng'] >= min_lng) & (rentdf['lng'] <= max_lng)]    
    filtered = rent_filter(rentdf=filtered_df, radius_km=radius_km, sale_row=sale_row)
    # display(filtered[:5])
    result = dict(min_price_sqm=filtered.price_sqm.min(),
                  mean_price_sqm=filtered.price_sqm.median(),
                  avg_price_sqm=filtered.price_sqm.mean(),
                  max_price_sqm=filtered.price_sqm.max(),
                  count=filtered.price_sqm.count(),
                  urls=filtered.url.to_list())
    result["mean_yield"] = result["mean_price_sqm"] * 12 / sale_row.price_sqm * 100
    result_str = " ".join([f"{k}={v}" for k, v in result.items()])
    if verbose:
        print(f"{sale_row.lat=} {lat_diff=} {min_lat=} {max_lat=}")
        print(f"{sale_row.lng=} {lng_diff=} {min_lng=} {max_lng=}")
        print(f"before:{len(rentdf)} after {len(filtered_df)}")
        print(result_str)
    return result

    

In [12]:
def enrich(df):
    resultdf = saledf.parallel_apply(lambda r: rent_finder(rentdf=rentdf, radius_km=1, sale_row=r, verbose=False), axis=1, result_type='expand')
    return pd.concat([df, resultdf], axis=1)

enriched = enrich(saledf)
enriched

,url,title,price,original_price,price_per_sqm,location,posted,ad_id,reference_number,views,lat,lng,sold,cat0,cat1,Property area,Pets,Type,Parking,Plot area,Furnishing,Included,Online viewing,Air conditioning,Energy Efficiency,Bedrooms,Bathrooms,images,description,Floor,Postal code,Construction year,Reference number,Condition,Square meter price,Minimum stay,Land type,Plot Type,Parcel number,Planning zone,Registration number,Share,Density,Coverage,Registration block,Area,Pick a point,price_sqm,city,district,min_price_sqm,mean_price_sqm,avg_price_sqm,max_price_sqm,count,urls,mean_yield
5,https://www.bazaraki.com/adv/5503436_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,376000,NaN,3082.0000,"Paphos, Paphos - Agios Theodoros",20.11.2024 10:09,5503436,NaN,312,34.7746,32.4224,False,Cyprus real estate for sale,"Apartments, flats for sale",122.0000,NaN,Apartment,Covered,NaN,Unfurnished,"Pool, Garden, Elevator, Balcony, Storage room",NaN,"Full, all rooms",In Progress,2,2,"[https://cdn1.bazaraki.com/media/cache1/89/9a/899aeafa91fac401f6ba6590d7cf0c15.webp, https://cdn...",2 and 3 bedrooms，communal GYM，rooftop swimming pool， underfloor heating,7th,NaN,NaN,NaN,Brand new,€3.082 /,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3081.9672,Paphos,Paphos - Agios Theodoros,6.9231,14.7727,206.8894,8828.1250,46,"[https://www.bazaraki.com/adv/5406774_3-bedroom-apartment-to-rent/, https://www.bazaraki.com/adv...",5.7519
849,https://www.bazaraki.com/adv/5321953_3-bedroom-apartment-for-sale/,3-bedroom apartment fоr sаle,178000,NaN,1548.0000,"Nicosia, Nicosia - Kaimakli",06.12.2024 12:00,5321953,NaN,658,35.1914,33.3828,False,Cyprus real estate for sale,"Apartments, flats for sale",115.0000,NaN,Apartment,Covered,NaN,Appliances οnly,"Elevator, Fireplace, Balcony, Storage room",No,No,B,3,2,"[https://cdn1.bazaraki.com/media/cache1/8f/aa/8faa9e576f91e6c849fffc575a3b4fb8.webp, https://cdn...","Brand new 3-bedroom apartment in Kaimakli, Nicosia\nNo VAT applies.\nSubsidy up to €65,000 appli...",3rd,1025.0000,2019,NaN,Brand new,€1.548 /,NaN,NaN,NaN,NaN,NaN,3.0000,NaN,NaN,NaN,2.0000,NaN,NaN,1547.8261,Nicosia,Nicosia - Kaimakli,3.7778,9.2857,8.8425,11.7021,8,"[https://www.bazaraki.com/adv/5561469_1-bedroom-apartment-to-rent/, https://www.bazaraki.com/adv...",7.1990
850,https://www.bazaraki.com/adv/5433785_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,480000,NaN,4034.0000,"Limassol, Agios Athanasios",06.12.2024 10:28,5433785,NaN,129,34.7139,33.0599,False,Cyprus real estate for sale,"Apartments, flats for sale",119.0000,NaN,Apartment,Covered,NaN,NaN,"Pool, Elevator, Alarm, Balcony, Playroom, Storage room",NaN,"Full, all rooms",A,2,2,"[https://cdn1.bazaraki.com/media/cache1/48/96/48965f146eec15aef11039592acbeb96.webp, https://cdn...",Sea Breeze 5 - Agios Athanasios\nConstruction Status – Under Construction\nLuxury 2 Bedroom Apar...,1st,NaN,NaN,NaN,Under construction,€4.034 /,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4033.6134,Limassol,Agios Athanasios,11.1111,20.4082,20.9428,32.4675,53,"[https://www.bazaraki.com/adv/5518960_3-bedroom-apartment-to-rent/, https://www.bazaraki.com/adv...",6.0714
853,https://www.bazaraki.com/adv/5326549_2-bedroom-penthouse-for-sale/,2-bedroom penthouse fоr sаle,310000,NaN,3780.0000,"Larnaca, Larnaka - Sotiros",06.12.2024 16:33,5326549,NaN,327,34.9362,33.6289,False,Cyprus real estate for sale,"Apartments, flats for sale",82.0000,NaN,Penthouse,Covered,NaN,Unfurnished,"Garden, Elevator, Balcony, Storage room",Yes,No,A,2,2,"[https://cdn1.bazaraki.com/media/cache1/c5/ce/c5ceb83a22f0fff9b25f0fe163c411e6.webp, https://cdn...",2 bedroom Luxury brand new Penthouse with Roof Garden\n>\tREADY TO MOVE IN and just delivered\n>...,4th,6057.0000,2024,NaN,Brand new,€3.780 /,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3780.4878,Larnaca,Larnaka - Sotiros,7.1429,13.8750,13.6170,22.6667,48,"[https://www.bazaraki.com/adv/4734916_2-bedroom-apartment-to-rent/, https://www.bazaraki.com/adv...",4.4042
854,https://www.bazaraki

In [13]:
enriched["mean_yield"].isna().mean()

np.float64(0.2873066959729963)

In [14]:
def report(fp, df):
    for index, row in enriched.sort_values("mean_yield", ascending=False).iterrows():
        row_str = f"{row.url} {row.city} price={row.price} area={row['Property area']} mean_yield={row['mean_yield']:.2f} max_price_sqm:{row.max_price_sqm:.2f} min_price_sqm={row.min_price_sqm:.2f} avg_price_sqm={row.avg_price_sqm:.2f} mean_price_sqm={row.mean_price_sqm:.2f} count={row['count']}"
        print(row_str, file=fp)
        for line in row.urls[:10]:
            print(f"\t{line}", file=fp)
            
with Path("output/report.txt").open("w") as fp:
    report(fp, enriched)

In [76]:
enriched.sort_values("yield", ascending=False).head(20)

,url,title,price,original_price,price_per_sqm,location,posted,ad_id,reference_number,views,lat,lng,sold,cat0,cat1,Property area,Pets,Type,Parking,Plot area,Furnishing,Included,Online viewing,Air conditioning,Energy Efficiency,Bedrooms,Bathrooms,images,description,Floor,Postal code,Construction year,Reference number,Condition,Square meter price,Minimum stay,Land type,Plot Type,Parcel number,Planning zone,Registration number,Share,Density,Coverage,Registration block,Area,Pick a point,price_sqm,city,district,max_price_sqm,min_price_sqm,avg_price_sqm,mean_price_sqm,count,urls,yield
24349,https://www.bazaraki.com/adv/5515485_3-bedroom-apartment-for-sale/,3-bedroom apartment fоr sаle,140000,NaN,1.0000,"Nicosia, Strovolos - Ap. Varnavas and Ag. Makar.",09.12.2024 20:31,5515485,NaN,282,35.1587,33.3441,False,Cyprus real estate for sale,"Apartments, flats for sale",130000.0000,NaN,Apartment,Covered,NaN,Unfurnished,"Elevator, Balcony, Storage room",Yes,"Full, all rooms",B,3,1,"[https://cdn1.bazaraki.com/media/cache1/d0/d8/d0d8278812e1db3cf1a537210cb66e24.webp, https://cdn...",BRIGHT AND SPACIOUS 3 BED FLAT FOR SALE IN STROVOLOS /AGIOI OMOLOGITES AREA\nOff prodromou stree...,1st,NaN,Older,NaN,Resale,€1 /,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0769,Nicosia,Strovolos - Ap. Varnavas and Ag. Makar.,15.9292,5.5556,9.3943,8.9899,52,"[https://www.bazaraki.com/adv/5536582_2-bedroom-apartment-to-rent/, https://www.bazaraki.com/adv...",10017.3160
14415,https://www.bazaraki.com/adv/5552333_3-bedroom-penthouse-for-sale/,3-bedroom penthouse fоr sаle,250000,NaN,1894.0000,"Nicosia, Agios Dometios - Agios Georgios",30.11.2024 12:19,5552333,NaN,26,35.1841,33.3382,False,Cyprus real estate for sale,"Apartments, flats for sale",132.0000,NaN,Penthouse,Covered,NaN,Unfurnished,NaN,NaN,"Full, all rooms",B,3,2,"[https://cdn1.bazaraki.com/media/cache1/fb/88/fb889de5fd593fc8b6ce06c37b84cde0.webp, https://cdn...","Experience elevated living in this modern penthouse, meticulously maintained and ready for you t...",NaN,NaN,NaN,9323,Resale,€1.894 /,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1893.9394,Nicosia,Agios Dometios - Agios Georgios,650.0000,5.0000,327.5000,327.5000,2,"[https://www.bazaraki.com/adv/5546166_3-bedroom-apartment-to-rent/, https://www.bazaraki.com/adv...",207.5040
5430,https://www.bazaraki.com/adv/5361364_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,192000,NaN,2370.0000,"Nicosia, Agios Dometios - Agios Georgios",20.11.2024 17:46,5361364,NaN,20,35.1866,33.3315,False,Cyprus real estate for sale,"Apartments, flats for sale",81.0000,NaN,Apartment,Covered,NaN,Unfurnished,NaN,NaN,No,A,2,2,"[https://cdn1.bazaraki.com/media/cache1/15/34/1534a3388ce8c0c2f464e096bed8c540.webp, https://cdn...",This unique project sets new standards in flat design and captures your attention with its simpl...,NaN,NaN,NaN,8914,Under construction,€2.370 /,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2370.3704,Nicosia,Agios Dometios - Agios Georgios,650.0000,5.0000,327.5000,327.5000,2,"[https://www.bazaraki.com/adv/5546166_3-bedroom-apartment-to-rent/, https://www.bazaraki.com/adv...",165.7969
983,https://www.bazaraki.com/adv/4842309_1-bedroom-apartment-for-sale/,1-bedroom apartment fоr sаle,290000,NaN,322.0000,"Limassol, Germasogeia",14.11.2024 17:36,4842309,NaN,46,34.6953,33.0866,False,Cyprus real estate for sale,"Apartments, flats for sale",900.0000,NaN,Apartment,Covered,NaN,Fully Furnished,"Storage room, Alarm",NaN,NaN,In Progress,1,NaN,[],"An outstanding studio apartment, on the second floor of a residential building. Designed and ins...",2nd,NaN,NaN,331149,Resale,€322 /,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,322.2222,Limassol,Germasogeia,108.4746,10.2804,28.8595,23.7288,179,"[https://www.bazaraki.com/adv/5469545_3-bedroom-apartment-to-rent/, https://www.bazaraki.com/adv...",88.3694
11109,https://www.bazaraki.com/adv/5494988_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,400000,NaN,530.0000,"Limassol, Germasogeia",25.11.20